# Overview

## Image Prediction
- I wrote a similar project for a class in college, but our data was a set of hand-drawn 8s and 9s.
- In college, my team would frequently sail at MIT, so we would use their boathouse cameras to determine the day's conditions.
- I have combined the underlying principles of my project with the data from the MIT cameras to predict whether there is a person on the dock or a sailboat in the water in any given still image.

## TODO
- Try different models besides just Random Forrest
- Generate signfiicantly more training data - currently about 50 examples
- Slightly reduce inline comment and provide more context at beginning of blocks
- Code method for reducing images from their original size to fit in 320x320 pixels (all images end up as 320x180), currently done manually after downloading them
- Determine more "target columns" and then make them variables//remove hard-coding in body of code
- Use preprocess_image() elsewhere if possible, maybe have it defined earlier
- Streamline code where possible - see what an LLM will say about it?
- Evaluate any time/space complexity hot spots


## Imports, Paths, Hard-Coded Variables


In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix


photo_folder_path = "./data/20230912/"
data_file_path    = "./MIT Sailing Camera Data.xlsx"
data_sheet_name   = "S Data 12 Sep 23"
data_labels_ind   = "Time https://sailing.mit.edu/eyesee/"
data_labels_to_use = [data_labels_ind, "sailboats_on_dock", "sailboats_on_dock", "sailboats_afloat", "powers_afloat", "people_visible",
                      # these below are the "y values"
                      "bool_people", "bool_sailboat_afloat"]


## Load Data
### Output: df_combined is a dataframe which has one column for each pixel in the images, as well as one column for each of the elements of the manual labeling from the Excel file.  It has the same number of rows as the number of rows in the Excel file, i.e. the number of labeled images.

#### Approx time: 0.75-1.5 minutes

In [2]:
# Read Excel
df_data_labels = pd.read_excel(data_file_path, sheet_name=data_sheet_name, usecols=data_labels_to_use)

# Convert data_labels_ind (the index) to string and drop rows missing index
df_data_labels[data_labels_ind] = df_data_labels[data_labels_ind].astype(str).str.strip()
df_data_labels = df_data_labels[df_data_labels[data_labels_ind].notna()]

# Construct photo_list and photo_ids from the information in Excel
# The images in the photo folder which should be considered all have rows in the Excel
photo_ids  = df_data_labels[data_labels_ind].tolist()
photo_list = [pid + ".jpg" for pid in photo_ids]

# Load images and flatten (only include those that exist)
image_data = []
valid_photo_ids = []

# photo_ids = list of ints
# photo_list = list of those ints with ".jpg" at end
for pid, photo in zip(photo_ids, photo_list):
    try:
        with Image.open(photo_folder_path + photo) as p:
            gray = p.convert("L")
            flat_array = np.array(gray).flatten()
            image_data.append(flat_array)
            valid_photo_ids.append(pid)  # Only keep if image was successfully opened
    except FileNotFoundError:
        print(f"Warning: {photo} not found. Skipping.")
        
        
# Create DataFrame from image data
df_images = pd.DataFrame(image_data, index=valid_photo_ids)

# Create df_filtered from df_data_labels
df_filtered = df_data_labels[df_data_labels[data_labels_ind].isin(valid_photo_ids)].copy()
df_filtered.set_index(data_labels_ind, inplace=True)

# Combine using aligned indexes
df_combined = pd.concat([df_images, df_filtered], axis=1)

# Ensure all column names are strings
df_combined.columns = df_combined.columns.astype(str)



In [3]:
# print(df_combined.shape)

In [4]:
# print(df_combined)

## Scale the data by column

In [5]:
# Apply MinMaxScaler column-wise
scaler = MinMaxScaler()

df_combined_scaled = pd.DataFrame(
    scaler.fit_transform(df_combined),
    columns=df_combined.columns.astype(str),
    index=df_combined.index
)

In [6]:
# print(df_combined_scaled)

## Create models to predict the values of "bool_people" and "bool_sailboat_afloat" when seeing a new image
- bool_people is true when there are any number of people on the dock in the image
- bool_sailboat_afloat is true when there are any number of sailboats on the water

In [7]:
target_columns = ["bool_people", "bool_sailboat_afloat"]

# Drop target columns before scaling
X = df_combined.drop(columns=target_columns)
y_people = df_combined["bool_people"]
y_sail   = df_combined["bool_sailboat_afloat"]

# Scale only input features
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

# Split Data
X_train, X_test, yp_train, yp_test = train_test_split(X_scaled, y_people, test_size=0.1, random_state=42)
_,           _,  ys_train, ys_test = train_test_split(X_scaled, y_sail,   test_size=0.1, random_state=42)

# Train Models
clf_people = RandomForestClassifier(random_state=42)
clf_people.fit(X_train, yp_train)

clf_sail = RandomForestClassifier(random_state=42)
clf_sail.fit(X_train, ys_train)

RandomForestClassifier(random_state=42)

# Predict on test sets

In [8]:
yp_pred = clf_people.predict(X_test)
ys_pred = clf_sail.predict(X_test)

# Evaluate bool_people
print("Evaluation: bool_people")
print(f"Accuracy: {accuracy_score(yp_test, yp_pred):.1f}")
print("Confusion Matrix:")
print(confusion_matrix(yp_test, yp_pred))


# Evaluate bool_sailboat_afloat
print("\nEvaluation: bool_sailboat_afloat")
print(f"Accuracy: {accuracy_score(ys_test, ys_pred):.1f}")
print("Confusion Matrix:")
print(confusion_matrix(ys_test, ys_pred))

print("\nConfusion Matrix:\nTP FN\nFP TN")



Evaluation: bool_people
Accuracy: 0.7
Confusion Matrix:
[[0 2]
 [0 4]]

Evaluation: bool_sailboat_afloat
Accuracy: 0.7
Confusion Matrix:
[[3 2]
 [0 1]]

Confusion Matrix:
TP FN
FP TN


In [9]:
def preprocess_image(img_path, scaler, input_columns):
    """
    Open and preprocess a new image into a scaled feature vector.

    Parameters:
    - img_path: path to new image
    - scaler: fitted MinMaxScaler (used only on input features)
    - input_columns: list of columns used during training (image + metadata, but excluding targets)

    Returns:
    - Scaled DataFrame matching X input used for training
    """
    with Image.open(img_path) as p:
        gray = p.convert("L")
        flat_array = np.array(gray).flatten()

    # Build DataFrame with image data (ensure correct col count)
    df_new = pd.DataFrame([flat_array], columns=input_columns[:len(flat_array)])

    # Fill any remaining Excel-origin columns with 0
    for col in input_columns[len(flat_array):]:
        df_new[col] = 0

    df_new = df_new[input_columns]  # ensure order
    df_new_scaled = pd.DataFrame(scaler.transform(df_new), columns=df_new.columns)

    return df_new_scaled

In [10]:
new_imgs = ["1400", "1407", "1419", "1428", "1658", "1702", "1727", "1759"]

for i in new_imgs:
    # Path to new image
    new_image_path = "./data/20230912/" + i + ".jpg"

    input_columns = X.columns.tolist()  # X already excludes target columns

    # Preprocess i
    df_new_scaled = preprocess_image(new_image_path, scaler, input_columns)

    # Predict
    pred_people = clf_people.predict(df_new_scaled)[0]
    pred_sail   = clf_sail.predict(df_new_scaled)[0]

    print(f"Prediction {i}: bool_people = {pred_people}, bool_sailboat_afloat = {pred_sail}")


Prediction 1400: bool_people = 1, bool_sailboat_afloat = 1
Prediction 1407: bool_people = 1, bool_sailboat_afloat = 1
Prediction 1419: bool_people = 1, bool_sailboat_afloat = 0
Prediction 1428: bool_people = 1, bool_sailboat_afloat = 0
Prediction 1658: bool_people = 1, bool_sailboat_afloat = 1
Prediction 1702: bool_people = 1, bool_sailboat_afloat = 1
Prediction 1727: bool_people = 1, bool_sailboat_afloat = 1
Prediction 1759: bool_people = 0, bool_sailboat_afloat = 0


In [11]:
# Image, People, Sailboat ... Prediction
# 1400, 1, 0 ... 1, 1 ... x
# 1407, 0, 0 ... 1, 1 ... x
# 1419, 1, 0 ... 1, 0 ... good
# 1428, 0, 0 ... 1, 1 ... x
# 1658, 1, 0 ... 1, 1 ... x
# 1702, 1, 1 ... 1, 1 ... good
# 1727, 1, 1 ... 1, 1 ... good
# 1759, 1, 1 ... 1, 0 ... x
